In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
dataframe = pd.read_csv("/content/drive/My Drive/ML Project Taxi Fair/train.csv")

In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
#Save dataset without nan values
dataframe.to_csv("/content/drive/My Drive/ML Project Taxi Fair/Train_without_nan.csv", index=False)

In [ ]:
dataframe.isna().sum()

tripid                       0
additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
dtype: int64

In [ ]:
#Try to fit the data without any transeformation

#Convert labels for 1 and 0
#Before do describe, Let's convert the label as 1 and 0 .(1-correct and 0-incorrect)
def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

dataframe['label']= dataframe['label'].apply(encoding_label).values

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


In [ ]:
X = dataframe.drop(columns=['label', 'tripid', 'drop_time', 'pickup_time']).values
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))

Accuracy score : 0.91101944608132


In [ ]:
predictions = model.predict(X_validation)

print("F1 score {0}".format(f1_score(predictions, Y_validation)))

F1 score 0.9510851959831552


In [ ]:
#So the raw data without any tranformations gives 0.910 accuracy and 0.95059 f1 score
#Let's transform the data by np.log because off high skewness

dataframe_tr = dataframe.drop(columns=['tripid','tripid', 'drop_time', 'pickup_time'])

In [ ]:
#For log scale the value can't be zero. So add 1 to every cell
dataframe_tr = dataframe_tr+1

#Convert to log scale
dataframe_tr = np.log(dataframe_tr)

In [ ]:
#Model fit and train
X = dataframe_tr.drop(columns=['label']).values
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = LogisticRegression()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9192692987625221
F1 score 0.9575062034739454


In [ ]:
#Lets deal with np.log and min max scaler

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = LogisticRegression()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9142604596346494
F1 score 0.9550717924965262


In [ ]:
#Lets deal with np.log and Standard scaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = LogisticRegression()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9189746611667649
F1 score 0.9573577298806016


In [ ]:
#Lets deal with np.log and Robust scaler

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = LogisticRegression()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9189746611667649
F1 score 0.9573577298806016


In [ ]:
#Lets choose np.log and minmax and Grid search for parameters

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler


parameters = {
     'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['saga']
    }
####Minmaxand log transformation start

scaler = MinMaxScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

####end of transformation

model = LogisticRegression()

clf = GridSearchCV(model, parameters)

# Fit the grid search
clf.fit(X_train, Y_train)

# View The Best Parameters
print('Best penalty:', clf.best_estimator_.get_params()['penalty'])
print('Best C:', clf.best_estimator_.get_params()['C'])
print("Best score : ", clf.best_score_)

#Predictions
model = clf.best_estimator_
predictions = model.predict(X_validation)
print("F1 Score : ", f1_score(Y_validation, predictions))
clf.best_estimator_


Best penalty: l1
Best C: 206.913808111479
Best score :  0.9170472727864928
F1 Score :  0.9573577298806016


LogisticRegression(C=206.913808111479, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)